<a href="https://colab.research.google.com/github/namanphy/END2/blob/main/S5/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up

In [2]:
# ! git clone https://github.com/namanphy/END2.git
# cd /content/END2/S7

In [1]:
import pandas as pd



---



## Loading Phrase-Label dictionary

In [43]:
dictionary = pd.read_csv("data/stanfordSentimentTreebank/dictionary.txt", sep="|", names=['phrase', 'phrase ids'] )

In [44]:
dictionary.head()

,phrase,phrase ids
0,!,0
1,! ',22935
2,! '',18235
3,! Alas,179257
4,! Brilliant,22936


In [45]:
dictionary.shape

(239232, 2)

## Loading labels

In [13]:
label_mapping = pd.read_csv("data/stanfordSentimentTreebank/sentiment_labels.txt",
                            sep="|")

In [38]:
label_mapping.head()

,phrase ids,sentiment values,sentiment labels
0,0,0.50000,2
1,1,0.50000,2
2,2,0.44444,2
3,3,0.50000,2
4,4,0.42708,2


In [39]:
label_mapping.shape

(239232, 3)

In [29]:
def discretize_label(label):
    if label <= 0.2: return 0
    if label <= 0.4: return 1
    if label <= 0.6: return 2
    if label <= 0.8: return 3
    return 4


label_mapping['sentiment labels'] = label_mapping['sentiment values'].apply(discretize_label)

In [30]:
label_mapping.head()

,phrase ids,sentiment values,sentiment labels
0,0,0.50000,2
1,1,0.50000,2
2,2,0.44444,2
3,3,0.50000,2
4,4,0.42708,2


In [40]:
assert label_mapping.shape[0] == dictionary.shape[0]

## Loading sentences

In [60]:
sentences = pd.read_csv("data/stanfordSentimentTreebank/datasetSentences.txt", sep="\t", 
                        names=['sentence ids', 'sentence'], skiprows=1)

In [61]:
sentences

,sentence ids,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,"Emerges as something rare , an issue movie tha..."
...,...,...
11850,11851,A real snooze .
11851,11852,No surprises .
11852,11853,We 've seen the hippie-turned-yuppie plot befo...
11853,11854,Her fans walked out muttering words like `` ho...


# Merging the dataframes

In [62]:
sentence_phrase_merge = pd.merge(sentences, dictionary, left_on='sentence', right_on='phrase')

In [63]:
df = pd.merge(sentence_phrase_merge, label_mapping, on='phrase ids')

In [64]:
df = df[['sentence', 'sentiment labels']]

In [65]:
df.head()

,sentence,sentiment labels
0,The Rock is destined to be the 21st Century 's...,3
1,The gorgeously elaborate continuation of `` Th...,4
2,Effective but too-tepid biopic,2
3,If you sometimes like to go to the movies to h...,3
4,"Emerges as something rare , an issue movie tha...",4


In [66]:
df['sentence_clean'] = df['sentence'].str.replace(r"\s('s|'d|'re|'ll|'m|'ve|n't)\b", lambda m: m.group(1))

In [68]:
df

,sentence,sentiment labels,sentence_clean
0,The Rock is destined to be the 21st Century 's...,3,The Rock is destined to be the 21st Century's ...
1,The gorgeously elaborate continuation of `` Th...,4,The gorgeously elaborate continuation of `` Th...
2,Effective but too-tepid biopic,2,Effective but too-tepid biopic
3,If you sometimes like to go to the movies to h...,3,If you sometimes like to go to the movies to h...
4,"Emerges as something rare , an issue movie tha...",4,"Emerges as something rare , an issue movie tha..."
...,...,...,...
11281,A real snooze .,0,A real snooze .
11282,No surprises .,1,No surprises .
11283,We 've seen the hippie-turned-yuppie plot befo...,3,We've seen the hippie-turned-yuppie plot befor...
11284,Her fans walked out muttering words like `` ho...,0,Her fans walked out muttering words like `` ho...


In [69]:
df.to_csv("sst_dataset_parsed.csv")

-----

In [1]:
from data.sst_dataset import sentences_to_dataframe

In [2]:
train, test = sentences_to_dataframe('./data/stanfordSentimentTreebank')

E:\code\my-code\END2\S7\data\sst_dataset.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  df['sentence_clean'] = df['sentence'].str.replace(r"\s('s|'d|'re|'ll|'m|'ve|n't)\b", lambda m: m.group(1))


In [3]:
train.shape

(9029, 3)

In [4]:
test.shape

(2257, 3)